## Домашнее задание 5. Логистическая регрессия и случайный лес в задаче кредитного скоринга. Решение

В этом задании вы построите модели и ответите на вопросы, используя данные по кредитному скорингу.

Начнём с разминочного упражнения.

**Вопрос 1.** В зале суда 5 присяжных. Каждый из них может правильно определить виновность подсудимого с вероятностью 70%, независимо друг от друга. Какова вероятность того, что присяжные совместно вынесут правильный вердикт, если окончательное решение принимается большинством голосов?

1. 70.00%
2. 83.20%
3. 83.70%
4. 87.50%

**Ответ:** 3.

### Решение:

Используем формулу для $\mu$ из статьи. Так как большинство голосов начинается с $3$, то $m = 3, ~N = 5, ~p = 0.7$. Подставим эти значения в формулу:

$$\large \mu = \sum_{i=3}^{5}{5 \choose i}0.7^i(1-0.7)^{5-i} = 83.70\%$$

## Постановка задачи кредитного скоринга

#### Задача

Предсказать, вернёт ли клиент кредит в течение 90 дней. Это задача бинарной классификации — мы будем относить клиентов к категориям «хороший» или «плохой» на основе нашего прогноза.

#### Описание данных

| Признак | Тип переменной | Тип значений | Описание |
|:--------|:--------------|:-----------|:----------|
| age | Входной признак | integer | Возраст клиента |
| DebtRatio | Входной признак | real | Общие ежемесячные платежи по кредитам / Общий ежемесячный доход в процентах |
| NumberOfTime30-59DaysPastDueNotWorse | Входной признак | integer | Количество случаев просрочки 30–59 дней (не хуже) за последние 2 года |
| NumberOfTimes90DaysLate | Входной признак | integer | Количество случаев просрочки 90+ дней |
| NumberOfTime60-89DaysPastDueNotWorse | Входной признак | integer | Количество случаев просрочки 60–89 дней (не хуже) за последние 2 года |
| NumberOfDependents | Входной признак | integer | Количество иждивенцев клиента |
| SeriousDlqin2yrs | Целевая переменная | binary: <br>0 или 1 | Клиент не погасил долг по кредиту в течение 90 дней |

Настроим окружение:

In [ ]:
# Отключение предупреждений
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8

Напишем функцию, которая заменит значения *NaN* медианой по каждому столбцу.

In [ ]:
def fill_nan(table):
    for col in table.columns:
        table[col] = table[col].fillna(table[col].median())
    return table   

Прочитаем данные:

In [ ]:
data = pd.read_csv('../data/credit_scoring_sample.csv')
data.head()

Посмотрим на типы переменных:

In [ ]:
data.dtypes

Проверим баланс классов:

In [ ]:
ax = data['SeriousDlqin2yrs'].hist(orientation='horizontal', color='red')
ax.set_xlabel("number_of_observations")
ax.set_ylabel("unique_value")
ax.set_title("Target distribution")

print('Distribution of the target:')
data['SeriousDlqin2yrs'].value_counts()/data.shape[0]

Выделим имена входных переменных, исключив целевую:

In [ ]:
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']
independent_columns_names

Применим функцию для замены значений *NaN*:

In [ ]:
table = fill_nan(data)

Разделим целевую переменную и входные признаки:

In [ ]:
X = table[independent_columns_names]
y = table['SeriousDlqin2yrs']

## Бутстрэп

**Вопрос 2.** Постройте интервальную оценку среднего возраста клиентов, просрочивших выплату, с уровнем доверия 90%. Используйте `np.random.seed(0)`. Какой получился доверительный интервал?

1. 52.59 – 52.86
2. 45.71 – 46.13
3. 45.68 – 46.17
4. 52.56 – 52.88

**Ответ:** 2.

### Решение:

In [ ]:
def get_bootstrap_samples(data, n_samples):
    """Генерация выборок с помощью бутстрэпа."""
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    """Интервальная оценка."""
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

# Сохраняем возраст клиентов с просрочкой
churn = data[data['SeriousDlqin2yrs'] == 1]['age'].values

# Устанавливаем seed для воспроизводимости
np.random.seed(0)

# Генерируем бутстрэп-выборки и считаем среднее для каждой
churn_mean_scores = [np.mean(sample) for sample in get_bootstrap_samples(churn, 1000)]

# Выводим интервальную оценку
print("Mean interval", stat_intervals(churn_mean_scores, 0.1))

## Логистическая регрессия

Настроим логистическую регрессию:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

Создадим модель `LogisticRegression` с `class_weight='balanced'` для компенсации несбалансированности классов.

In [ ]:
lr = LogisticRegression(random_state=5, class_weight='balanced')

Попробуем найти лучший коэффициент регуляризации — параметр `C` логистической регрессии.

In [ ]:
parameters = {'C': (0.0001, 0.001, 0.01, 0.1, 1, 10)}

Для нахождения оптимального значения `C` используем стратифицированную 5-fold кросс-валидацию и посмотрим на *ROC AUC* при различных значениях `C`. Используем `StratifiedKFold`:

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

Одна из важных метрик качества модели — *Area Under the Curve (AUC)*. *ROC AUC* принимает значения от 0 до 1. Чем ближе ROC AUC к 1, тем лучше качество классификационной модели.

**Вопрос 3.** Выполните *Grid Search* с метрикой "roc_auc" по параметру `C`. Какое значение `C` оптимально?

1. 0.0001
2. 0.001
3. 0.01
4. 0.1
5. 1
6. 10

**Ответ:** 2.

### Решение:

In [ ]:
grid_search = GridSearchCV(lr, parameters, n_jobs=-1, scoring='roc_auc', cv=skf)
grid_search = grid_search.fit(X, y)
grid_search.best_estimator_

**Вопрос 4.** Можно ли считать лучшую модель стабильной? Модель *стабильна*, если стандартное отклонение на валидации менее 0.5%. Сохраните значение *ROC AUC* лучшей модели — оно понадобится далее.

1. Да
2. Нет

**Ответ:** 2.

### Решение:

In [ ]:
grid_search.cv_results_['std_test_score'][1]

Значение *ROC AUC* лучшей модели:

In [ ]:
grid_search.best_score_

## Важность признаков

**Вопрос 5.** *Важность признака* определяется абсолютным значением его соответствующего коэффициента. Сначала нужно нормализовать все значения признаков, чтобы их можно было корректно сравнивать. Какой признак наиболее важен для лучшей модели логистической регрессии?

1. age
2. NumberOfTime30-59DaysPastDueNotWorse
3. DebtRatio
4. NumberOfTimes90DaysLate
5. NumberOfTime60-89DaysPastDueNotWorse
6. MonthlyIncome
7. NumberOfDependents

**Ответ:** 2.

### Решение:

In [ ]:
from sklearn.preprocessing import StandardScaler
lr = LogisticRegression(C=0.001, random_state=5, class_weight='balanced')
scal = StandardScaler()
lr.fit(scal.fit_transform(X), y)

pd.DataFrame({'feat': independent_columns_names,
              'coef': lr.coef_.flatten().tolist()}).sort_values(by='coef', ascending=False)

**Вопрос 6.** Рассчитайте, насколько *DebtRatio* влияет на предсказание, используя [функцию softmax](https://en.wikipedia.org/wiki/Softmax_function). Какое значение получается?

1. 0.38
2. -0.02
3. 0.11
4. 0.24

**Ответ:** 3.

### Решение:

In [ ]:
print((np.exp(lr.coef_[0]) / np.sum(np.exp(lr.coef_[0])))[2])

**Вопрос 7.** Пересчитайте логистическую регрессию с абсолютными значениями (без масштабирования). Увеличьте возраст клиента на 20 лет, оставив остальные признаки без изменений. Во сколько раз увеличится вероятность того, что клиент не вернёт долг? Пример теоретического расчёта можно найти [здесь](https://www.unm.edu/~schrader/biostat/bio2/Spr06/lec11.pdf).

1. -0.01
2. 0.70
3. 8.32
4. 0.66

**Ответ:** 2.

### Решение:

In [ ]:
lr = LogisticRegression(C=0.001, random_state=5, class_weight='balanced')
lr.fit(X, y)

pd.DataFrame({'feat': independent_columns_names,
              'coef': lr.coef_.flatten().tolist()}).sort_values(by='coef', ascending=False)

In [ ]:
np.exp(lr.coef_[0][0]*20)

Вероятность того, что клиент не вернёт долг, изменяется в $\exp^{\beta\delta}$ раз, где $\delta$ — приращение значения признака. Это значит, что если увеличить возраст на 20 лет, шансы невозврата увеличатся в 0.69 раза (то есть фактически уменьшатся).

## Случайный лес

Импортируем классификатор случайного леса:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Инициализируем случайный лес со 100 деревьями и балансировкой целевых классов:

In [ ]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42, 
                            class_weight='balanced')

Будем искать лучшие параметры среди следующих значений:

In [ ]:
parameters = {'max_features': [1, 2, 4], 'min_samples_leaf': [3, 5, 7, 9], 'max_depth': [5,10,15]}

Снова используем стратифицированную кросс-валидацию. Переменная `skf` должна быть ещё доступна.

**Вопрос 8.** Насколько *ROC AUC* лучшей модели случайного леса выше, чем у лучшей логистической регрессии на валидации?

1. 4%
2. 3%
3. 2%
4. 1%

**Ответ:** 1.

### Решение:

In [ ]:
%%time
rf_grid_search = GridSearchCV(rf, parameters, n_jobs=-1, scoring='roc_auc', cv=skf, verbose=True)
rf_grid_search = rf_grid_search.fit(X, y)
print(rf_grid_search.best_score_ - grid_search.best_score_)

**Вопрос 9.** Какой признак имеет наименьшее влияние в модели случайного леса?

1. age
2. NumberOfTime30-59DaysPastDueNotWorse
3. DebtRatio
4. NumberOfTimes90DaysLate
5. NumberOfTime60-89DaysPastDueNotWorse
6. MonthlyIncome
7. NumberOfDependents

**Ответ:** 7.

### Решение:

In [ ]:
independent_columns_names[np.argmin(rf_grid_search.best_estimator_.feature_importances_)]

Рейтинг важности признаков:

In [ ]:
pd.DataFrame({'feat': independent_columns_names,
              'coef': rf_grid_search.best_estimator_.feature_importances_}).sort_values(by='coef', ascending=False)

**Вопрос 10.** Какое главное преимущество *логистической регрессии* перед *случайным лесом* в данной задаче?

1. Меньше времени на обучение модели
2. Меньше переменных для перебора
3. Интерпретируемость признаков
4. Линейные свойства алгоритма

**Ответ:** 3.

### Решение:

С одной стороны, модель случайного леса работает лучше для нашей задачи кредитного скоринга. Её качество выше на 4%. Причина — малое количество признаков и композиционное свойство случайных лесов.

С другой стороны, главное преимущество логистической регрессии — возможность интерпретировать влияние признаков на результат модели.

## Бэггинг

Импортируем модули и настроим параметры для бэггинга:

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

parameters = {'max_features': [2, 3, 4], 'max_samples': [0.5, 0.7, 0.9], 
              'estimator__C': [0.0001, 0.001, 0.01, 1, 10, 100]}

**Вопрос 11.** Обучите бэггинг-классификатор с `random_state=42`. В качестве базовых классификаторов используйте 100 логистических регрессий и `RandomizedSearchCV` вместо `GridSearchCV`. Установите максимальное число итераций равным 20. Не забудьте указать параметры `cv` и `random_state=1`. Какое лучшее значение *ROC AUC* вы получили?

1. 80.75%
2. 80.12%
3. 79.62%
4. 76.50%

**Ответ:** 1.

### Решение:

In [ ]:
bg = BaggingClassifier(LogisticRegression(class_weight='balanced'),
                       n_estimators=100, n_jobs=-1, random_state=42)
r_grid_search = RandomizedSearchCV(bg, parameters, n_jobs=-1, 
                                   scoring='roc_auc', cv=skf, n_iter=20, random_state=1,
                                   verbose=True)
r_grid_search = r_grid_search.fit(X, y)

In [ ]:
r_grid_search.best_score_

In [ ]:
r_grid_search.best_estimator_

**Вопрос 12.** Дайте интерпретацию лучших параметров бэггинга. Почему именно такие значения `max_features` и `max_samples` оптимальны?

1. Для бэггинга важно использовать как можно меньше признаков
2. Бэггинг работает лучше на малых выборках
3. Меньше корреляция между отдельными моделями
4. Чем больше признаков, тем меньше потеря информации

**Ответ:** 3.

### Решение:

Преимущество *случайного леса* в том, что деревья в композиции слабо коррелируют между собой. Аналогично, для бэггинга с логистической регрессией: чем слабее корреляция между отдельными моделями, тем выше точность. Поскольку в логистической регрессии практически нет случайности, нужно варьировать набор признаков, чтобы минимизировать корреляцию между отдельными моделями.